In [14]:
import subprocess
import time
import csv
import optuna
import pandas as pd

PARAMS_PATH = 'results/ma/ma_params.csv'
RESULTS_PATH = 'results/ma/ma_results.csv'

params = ['slow_ma', 'fast_ma']

def add_params(slow_ma, fast_ma):
    with open(PARAMS_PATH, mode='w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(params)
        writer.writerow([slow_ma, fast_ma])

def read_scores(slow_ma, fast_ma):
    tmp = pd.read_csv(RESULTS_PATH).query(f"slow_ma == {slow_ma} and fast_ma == {fast_ma}")
    while tmp.shape[0] < 1:
        tmp = pd.read_csv(RESULTS_PATH).query(f"slow_ma == {slow_ma} and fast_ma == {fast_ma}")
        time.sleep(0.1)
    return tmp['stats_mm'].mean()

def objective(trial):

    slow_ma = trial.suggest_int('slow_ma', 10, 100)
    fast_ma = trial.suggest_int('fast_ma', 1, 50)

    add_params(slow_ma, fast_ma)
    subprocess.check_call(["Rscript","helpers/run_strat_ma.R"],shell=False,stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    score = read_scores(slow_ma, fast_ma)
    return score

In [15]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10, show_progress_bar=True, study_name='Moving Average Strategy')
print(study.best_params)

[I 2025-01-17 22:59:03,787] A new study created in memory with name: no-name-0a0891a1-0057-4ff3-97f8-7572d3d0d751


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2025-01-17 22:59:04,370] Trial 0 finished with value: 1.4723974209607695 and parameters: {'slow_ma': 54, 'fast_ma': 3}. Best is trial 0 with value: 1.4723974209607695.
[I 2025-01-17 22:59:04,913] Trial 1 finished with value: 2.4910173802664457 and parameters: {'slow_ma': 98, 'fast_ma': 23}. Best is trial 1 with value: 2.4910173802664457.
[I 2025-01-17 22:59:05,461] Trial 2 finished with value: -0.8662918123490476 and parameters: {'slow_ma': 19, 'fast_ma': 42}. Best is trial 1 with value: 2.4910173802664457.
[I 2025-01-17 22:59:06,009] Trial 3 finished with value: 3.6891207377284183 and parameters: {'slow_ma': 66, 'fast_ma': 20}. Best is trial 3 with value: 3.6891207377284183.
[I 2025-01-17 22:59:06,536] Trial 4 finished with value: 1.0753807263782156 and parameters: {'slow_ma': 56, 'fast_ma': 3}. Best is trial 3 with value: 3.6891207377284183.
[I 2025-01-17 22:59:07,074] Trial 5 finished with value: -1.881287835772935 and parameters: {'slow_ma': 15, 'fast_ma': 47}. Best is trial 3 w